<a href="https://colab.research.google.com/github/zanuarts/dicoding-BPML/blob/master/dicoding_15_TFDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# THIS NOTEBOOK IS STILL ERROR. KINDLY PLEASE CHECK THE ORIGINAL LESSON ON DICODING

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [ ]:
url = 'https://raw.githubusercontent.com/natashayulian/diamond_dataset/master/diamonds.csv'
df = pd.read_csv(url)
df.head

<bound method NDFrame.head of        Unnamed: 0  carat        cut color  ... price     x     y     z
0               1   0.23      Ideal     E  ...   326  3.95  3.98  2.43
1               2   0.21    Premium     E  ...   326  3.89  3.84  2.31
2               3   0.23       Good     E  ...   327  4.05  4.07  2.31
3               4   0.29    Premium     I  ...   334  4.20  4.23  2.63
4               5   0.31       Good     J  ...   335  4.34  4.35  2.75
...           ...    ...        ...   ...  ...   ...   ...   ...   ...
53935       53936   0.72      Ideal     D  ...  2757  5.75  5.76  3.50
53936       53937   0.72       Good     D  ...  2757  5.69  5.75  3.61
53937       53938   0.70  Very Good     D  ...  2757  5.66  5.68  3.56
53938       53939   0.86    Premium     H  ...  2757  6.15  6.12  3.74
53939       53940   0.75      Ideal     D  ...  2757  5.83  5.87  3.64

[53940 rows x 11 columns]>

In [ ]:
train, test = train_test_split(df, test_size = 0.2)
train, val = train_test_split(train, test_size = 0.2)

print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

34521 train examples
8631 validation examples
10788 test examples


In [ ]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = df.copy()
    labels = dataframe.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
      ds = ds.shuffle(buffer_size = len(dataframe))
    ds = ds.batch(batch_size)
    return ds
batch_size = 10
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [ ]:
# numeric column
carat = feature_column.numeric_column('carat')

In [ ]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
batch_size = 32
train_ds = df_to_dataset(train, batch_size = batch_size)
val_ds = df_to_dataset(val, shuffle = False, batch_size = batch_size)
test_ds = df_to_dataset(test, shuffle = False, batch_size = batch_size)

In [ ]:
model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(128, activation = 'relu'),
    layers.Dense(128, activation = 'relu'),
    layers.Dropout(.1),
    layers.Dense(1)
])

model.compile(optimizer = 'adam',
              loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics = ['accuracy'])

model.fit(train_ds,
          validation_data = val_ds,
          epochs = 10)